<a href="https://colab.research.google.com/github/MuriloMatos552525/JavaAno1/blob/main/SPRINT4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# <font color='red' style='font-size: 30px;'>*SPRINT* 4 - *HAALKTECH*</font>



## Desenvolvedores:

* ALECK CAPPUCCI – RM551340
* GABRIEL LIMA – RM99743
* GIOVANNA ALVAREZ – RM98892
* MURILO MATOS – RM552525 – (1TDSPF)
* SARAH ROSA – RM98482


<hr style='border: 2px solid red;'>



## **Dataset: https://www.kaggle.com/datasets/tysonpo/bike-ads-images-prices-specifications**

# **Descrição do dataset:**

Este Dataset abrange 10.052 anúncios de bicicletas, proveniente do
website: “Bike Exchange”, com 1.982 anúncios
Os dados incluem informações como título, imagem, preço, condição e
especificações das bicicletas, com ênfase em "Road Bikes" e "Mountain Bikes".
Importante destacar que as imagens do Bike Exchange apresentam maior qualidade
Medidas de filtragem foram aplicadas para remover anúncios não
relacionados a bicicletas.

# **Objetivo:**
As especificações do tipo, marca e entre outras,
devem ser inseridas em nosso banco de dados como uma base de
dados para inspeção automatizada. Em outras palavras, quando um
usuário realizar a vistoria automatizada e seguir as instruções exibidas
na tela enquanto estiver a capturar fotos da bicicleta, a IA será capaz
de identificar tanto o tipo, quanto a marca e o modelo da bicicleta que
está sendo fotografado pelo usuário. Portanto, caso o usuário tente
fornecer informações incorretas sobre o tipo de bicicleta que possui, a
IA será capaz de determinar com precisão o tipo real da bicicleta.
<hr style='border: 2px solid red;'>


Nesta célula, as bibliotecas necessárias são importadas e os dados são carregados a partir de arquivos JSON e CSV. Os dados são então combinados com base no ID.

In [ ]:
# Importe as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Carregar o conjunto de dados "data_bike_exchange.json"
data_bike_exchange = pd.read_json('data_bike_exchange.json', lines=True)
combined_price = pd.read_csv('combined_price-only.csv')
# Obs: Os dados do data_ebay.json não foram utilizados pois apresentavam diversas falhas.

# Combinar os dados com base no ID
data = data_bike_exchange.merge(combined_price, on='ID', how='inner')


Nesta célula, ocorre a limpeza e a conversão das colunas para tipos numéricos, juntamente com a remoção de linhas com valores ausentes na variável alvo.

In [ ]:
# Limpeza das colunas 'Frame in inches' e 'Size'
data['Frame in inches'] = data['Frame in inches'].str.replace('"', '', regex=True)
data['Size'] = data['Size'].str.replace('"', '', regex=True)

# Converta as colunas para tipo numérico
data['Frame in inches'] = pd.to_numeric(data['Frame in inches'], errors='coerce')
data['Size'] = pd.to_numeric(data['Size'], errors='coerce')

# Limpeza da coluna 'Price now'
data['Price now'] = data['Price now'].str.replace('$', '', regex=True)
data['Price now'] = data['Price now'].str.replace(',', '', regex=True)

# Converta a coluna 'Price now' para tipo numérico
data['Price now'] = pd.to_numeric(data['Price now'], errors='coerce')

# Remova linhas com valores ausentes na variável alvo
data.dropna(subset=['Price now'], inplace=True)


Nesta célula, é definida a classe alvo com base na mediana dos preços.

In [ ]:
# Defina uma classe alvo para classificação
data['Price Class'] = (data['Price now'] > data['Price now'].median()).astype(int)


Nesta célula, é definido o pré-processamento dos dados, incluindo a identificação de recursos categóricos e numéricos, a criação de um imputador para preencher valores ausentes e a configuração do transformador de colunas.

In [ ]:
# Pré-processamento de dados
categorical_features = ['Brand', 'Type', 'Color', 'Gender', 'Riding Style', 'Material', 'Wheel Size', 'Rear Derailleur', 'Item condition', 'Availability']
numeric_features = ['Frame in inches', 'Size']

# Imputador para preencher valores ausentes com a média
imputer = SimpleImputer(strategy='mean')

# Usando handle_unknown='ignore' ao criar o codificador OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('num', imputer, numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])


Nesta célula, os dados são divididos em conjuntos de treinamento e teste.

In [ ]:
X = data.drop(['Price now', 'Price Class'], axis=1)  # Recursos (removendo a coluna "Price now" e "Price Class")
y = data['Price Class']  # Variável alvo (classificação binária)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Nesta célula, é criado um pipeline que combina o pré-processamento de dados e o modelo de classificação (Logistic Regression).

In [ ]:
classification_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])


Nesta célula, o modelo é treinado com os dados de treinamento e as previsões são feitas no conjunto de teste.

In [ ]:
classification_pipeline.fit(X_train, y_train)

y_pred = classification_pipeline.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Nesta célula, são avaliados os resultados do modelo, incluindo a precisão, a matriz de confusão e o relatório de classificação.

In [ ]:

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)


print(f"Acurácia: {accuracy}")
print("Matriz de Confusão:")
print(conf_matrix)
print("Relatório de Classificação:")
print(class_report)


Acurácia: 0.8411214953271028
Matriz de Confusão:
[[131  33]
 [ 18 139]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.88      0.80      0.84       164
           1       0.81      0.89      0.84       157

    accuracy                           0.84       321
   macro avg       0.84      0.84      0.84       321
weighted avg       0.84      0.84      0.84       321



salvando o modelo em pickle

In [ ]:
with open('modelo.pickle', 'wb') as file:
    pickle.dump(classification_pipeline, file)

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict_price():
    try:
        data = request.json
        prediction = classification_pipeline.predict([data])
        return jsonify({'predicted_price': prediction[0]})
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
